In [1]:
import numpy as np, matplotlib.pyplot as plt
from anianssonwall.micellethermo import *

Simple sketch to illustrate the qualitative shape of the micelle energy landscape.

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(3.375, 3.375), sharex=True,
                                gridspec_kw={'hspace': 0.03})

length = 5
ratio = 1/length
free = 0.01
n_mode = 30
g0 = 10
nmax = 55

h, g, e = optimise_parameters(ratio, free, n_mode, g0=g0)

first = True
for de, fmt, label in zip([0.0425, e_crit(h, g) - e],
                            ['-', '-.'],
                            [r'$c_1 > \text{CMC}$', r'$c_1 \simeq \text{CMC}$']):
    model = MicelleDistribution(h, g, e + de)

    n = model.n
    n = np.linspace(1, n[-1], 1000)
    ceq = model.steady_state(n)
    n1, n2 = model.local_maximum, model.local_minimum

    pl, = ax1.plot(n, model.free_energy(n), fmt, label=label)
    G1, G2 = [model.free_energy(nn) for nn in [n1, n2]]
    # ax1.plot([n1, n2], [G1, G2], 'o', c=pl.get_color(), mfc='None')

    pl, = ax2.plot(n, ceq, fmt, label=label)
    c1, c2 = [model.steady_state(nn) for nn in [n1, n2]]
    # ax2.plot([n1, n2], [c1, c2], 'o', c=pl.get_color(), mfc='None')

    if first:
        for ax in [ax1, ax2]: ax.axvline(x=n1, ls='--', lw=0.5)

        bbox = bbox=dict(pad=0, fc='None', ec='None')

        na = 1 + 0.1*(n1-1)
        ax1.annotate(r'$\sim n^{2/3}$',
                        [na, model.free_energy(na)], [2*n1, 0.7],
                        ha='center', va='center', bbox=bbox,
                        arrowprops=dict(arrowstyle='-', lw=0.5))
        na = n1 + 0.5*(n2-n1)
        ax1.text(na, 1.4*model.free_energy(na), r'$\sim -n$',
                    ha='center', va='center', rotation=-40)
        na = n2 + 0.5*(n2-n1)
        ax1.text(na, 1.4*model.free_energy(na), r'$\sim n^{5/3}$',
                    ha='center', va='center', rotation=45)

        ax2.text(n1*1.1, 0.5, '$n^*$', ha='left', va='center', rotation=90)

        ax2.annotate(r'$C_\text{free}$', [0.5*n1, 0.6], [1.5*n1, 0.85],
                        bbox=bbox, arrowprops=dict(arrowstyle='-', lw=0.5))

        y = 0.25
        ax2.text(n2, y, r'$C_\text{mic}$', ha='center', va='center',
                    bbox=dict(pad=5, fc='w', ec='None'))
        ax2.annotate('', [n1, y], [nmax, y],
                        arrowprops=dict(arrowstyle='<->', lw=0.5), zorder=-1)

        first = False

for ax in [ax1, ax2]:
    ax.set_xticks([])
    ax.set_yticks([])

ax1.legend(loc='best')

ax1.set_ylabel(r'$W_n = G_n^\text{ex} - n G_1$')
ax2.set_ylabel(r'$c_n / c_1$')
ax2.set_xlabel(r'aggregate size $n$')

ax1.set_xlim([0, nmax])
ax1.set_ylim([0, 7])
ax2.set_ylim([0, 1])

label = ax1.text(0.025, 0.8, r'\textbf{a}', transform=ax1.transAxes,
                fontsize=18, ha='left', va='bottom')
label.set_in_layout(False)
label = ax2.text(0.975, 0.8, r'\textbf{b}', transform=ax2.transAxes,
                fontsize=18, ha='right', va='bottom')
label.set_in_layout(False)

plt.show()

Demonstrate how to calculate realistic equilibrium micelle distribution using the thermodynamic model of Maibaum et al. (2004).

In [ ]:
free = 0.01
n_mode = 30

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(3.375, 3.375), sharex=True)

for length, g0 in zip([2, 5, 7], [20, 10, 3.5]):
    label = '1:{}'.format(length)
    ratio = 1/length

    h, g, e = optimise_parameters(ratio, free, n_mode, g0=g0)
    tension = effective_tension_from_g(g, ratio)

    n = n_trial(h, g, e)
    n_smooth = np.linspace(1, n[-1], 1000)

    # ax1.plot(n_smooth, free_energy(n_smooth, h, g, e))
    pl, = ax1.step(n, free_energy(n, h, g, e), where='mid', label=label)
    n1, n2 = round(local_maximum(h, g, e)), round(local_minimum(h, g, e))
    G1, G2 = [free_energy(nn, h, g, e) for nn in [n1, n2]]
    ax1.plot([n1, n2], [G1, G2], 'o', c=pl.get_color(), mfc='None')

    G = free_energy(n, h, g, e)
    pl, = ax2.step(n, np.exp(-G), where='mid', label=label)
    c1, c2 = [np.exp(-free_energy(nn, h, g, e)) for nn in [n1, n2]]
    ax2.plot([n1, n2], [c1, c2], 'o', c=pl.get_color(), mfc='None')

for ax in [ax1, ax2]: ax.axvline(x=6, ls='--')

ax2.legend(loc='best')
ax1.set_ylabel(r'$\Delta G^\mathrm{ex}$')
ax2.set_ylabel(r'$c^\mathrm{eq}$')
ax2.set_xlabel(r'$n$')

# ax1.set_xlim([0, n[-1]])
ax1.set_xlim([0, 50])
# ax2.set_ylim([0, 1])

plt.show()

Demonstrate how to calculate realistic equilibrium micelle distribution using the thermodynamic model of Maibaum et al. (2004). This example is very similar to the previous one, but adjusts the distribution to give a desired absolute concentration. It also demonstrates how to use the object-orientated interface.

In [ ]:
free = 0.01
n_mode = 30
density = 1e-2

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(3.375, 3.375), sharex=True)

for length, g0 in zip([2, 5, 7], [20, 10, 3.5]):
    label = '1:{}'.format(length)
    ratio = 1/length

    model = MicelleDistribution.from_fit(ratio, free, n_mode,
                                            density=density, g0=g0)
    n = model.n
    ceq = model.steady_state(n)
    n1, n2 = round(model.local_maximum), round(model.local_minimum)

    # n_smooth = np.linspace(1, n[-1], 1000)
    # pl, = ax1.plot(n_smooth, model.free_energy(n_smooth))
    pl, = ax1.step(n, model.free_energy(n), where='mid', label=label)
    G1, G2 = [model.free_energy(nn) for nn in [n1, n2]]
    ax1.plot([n1, n2], [G1, G2], 'o', c=pl.get_color(), mfc='None')

    pl, = ax2.step(n, ceq, where='mid', label=label)
    c1, c2 = [model.steady_state(nn) for nn in [n1, n2]]
    ax2.plot([n1, n2], [c1, c2], 'o', c=pl.get_color(), mfc='None')

for ax in [ax1, ax2]: ax.axvline(x=6, ls='--')
ax2.legend(loc='best')

ax1.set_ylabel(r'$\Delta G_n$')
ax2.set_ylabel(r'$c^\mathrm{eq}$')
ax2.set_xlabel(r'$n$')

ax1.set_xlim([0, 50])

plt.show()

Demonstrate how to calculate the critical micelle concentration using the thermodynamic model of Maibaum et al. (2004).

In [ ]:
ratio = 0.2
free = 0.01
n_mode = 20,
density = 0.01
g0 = 0.5

h, g, e = optimise_parameters(ratio, free, n_mode, g0=g0)
n_crit = round(local_maximum(h, g, e))

# Extract the excess part of e using the density
n = n_trial(h, g, e)
G = free_energy(n, h, g, e)
unscaled_c = np.exp(-G)
unscaled_density = np.sum(n * unscaled_c)
# Rescale to achieve correct density.
ref_free_monomer_concentration = density / unscaled_density
kinetic_length = ref_free_monomer_concentration**(-1/3)
e_id = -np.log(ref_free_monomer_concentration * kinetic_length**3)
# Subtract ideal part to find excess.
e_ex = e + e_id

# c0 = ref_free_monomer_concentration * np.exp(-free_energy(n, h, g, e))
# plt.step(n, c0, where='mid')

# rho1 = 1.2*ref_free_monomer_concentration
# e_id = -np.log(rho1 * kinetic_length**3)
# c1 = rho1 * np.exp(-free_energy(n, h, g, e_ex - e_id))
# plt.step(n, c1, where='mid')

def run_calcs(rho1):
    e_id = -np.log(rho1 * kinetic_length**3)
    c = rho1 * np.exp(-free_energy(n, h, g, e_ex - e_id))

    # Work out concentration of free surfactant and micelles.
    cf, ct = c.copy(), c.copy()
    cf[n_crit:] = 0
    c_free, c_total = cf.dot(n), ct.dot(n)
    p_free = c_free / c_total

    return c_free, c_total, p_free

free_monomer_concentration = np.geomspace(1e-1, 1.25, 1000)
free_monomer_concentration *= ref_free_monomer_concentration
c_free = np.zeros_like(free_monomer_concentration)
c_total = np.zeros_like(free_monomer_concentration)
p_free = np.zeros_like(free_monomer_concentration)

for i, rho1 in enumerate(free_monomer_concentration):
    c_free[i], c_total[i], p_free[i] = run_calcs(rho1)

# Find critical micelle concentration.
e_cmc = e_crit(h, g)

e_id_cmc = e_ex - e_cmc
rho1_cmc = np.exp(-e_id_cmc) / kinetic_length**3
c_free_cmc, c_total_cmc, p_free_cmc = run_calcs(rho1_cmc)

plt.figure()
pl, = plt.plot(c_total, c_free, '-')
plt.plot(c_total_cmc, c_free_cmc, 'o', mfc='None', c=pl.get_color())
plt.xscale('log')
plt.yscale('log')
plt.plot(c_total, c_total, '--', c=pl.get_color(), scaley=False)
plt.xlabel('$c_\\mathrm{total}$')
plt.ylabel('$c_\\mathrm{free}$')

plt.show()